In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Imputer

In [2]:
spark = SparkSession.builder\
.appName("Module 2- Data_cleanin_and_Transformation")\
.enableHiveSupport()\
.master('yarn')\
.getOrCreate()

25/08/23 16:50:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
hdfs_path= '/tmp/Brazilian'

In [4]:
customers_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_customers_dataset.csv')

geolocation_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_geolocation_dataset.csv')

order_items_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_items_dataset.csv')

order_payments_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_payments_dataset.csv')

order_reviews_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_reviews_dataset.csv')

orders_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_orders_dataset.csv')

products_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_products_dataset.csv')

sellers_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_sellers_dataset.csv')


In [5]:
customers_df.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [6]:
#for missiong values 

def missing_values(dataframe,name):
    print(f'Missing Values in : {name}')
    dataframe.select([count(when(col(c).isNull(),1)).alias(c) for c in dataframe.columns]).show()

    
dataframes = [customers_df,geolocation_df,order_items_df,order_payments_df,order_reviews_df,orders_df,products_df,sellers_df]
name = ["customers_df","geolocation_df","order_items_df","order_payments_df","order_reviews_df","orders_df","products_df","sellers_df"]
for i in range(0,len(dataframes)):
    missing_values(dataframes[i],name[i])
    
#customers_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customers_df.columns]).show()

Missing Values in : customers_df
+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+

Missing Values in : geolocation_df


+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+

Missing Values in : order_items_df
+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
|       0|            0|         0|        0|                  0|    0|            0|
+--------+-------------+----------+---------+-------------------+-----+-------------+

Missing Values in : order_payments_df
+--------+

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|        1|    2236|        2380|               92157|                 63079|                8764|                   8785|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+

Missing Values in : orders_df
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+--------

# Handling Missing Values 

1\. Drop Missing Values (for non-Critical Columns )

  

2\. Fill missing values (for numerical columns)

  

3\. Impute Missing Values (for continuous data)

In [7]:
# droping the missing  calumns  
order_df_cleaned  = orders_df.na.drop(subset=['order_id','customer_id','order_status']) 

In [8]:
# filling the missing values 
orders_df_cleaned = orders_df.fillna({'order_delivered_customer_date':'9999-12-31'})

In [9]:
orders_df_cleaned.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

# Impute Missing Values 

In [10]:
from pyspark.ml.feature import Imputer

In [11]:
## cheking for null values 
temp = order_payments_df.withColumn('nullcheck',col('payment_value').isNull())

In [12]:
temp.filter( col('nullcheck')==True).show()

+--------+------------------+------------+--------------------+-------------+---------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|nullcheck|
+--------+------------------+------------+--------------------+-------------+---------+
+--------+------------------+------------+--------------------+-------------+---------+



### as we can see that there are no null values to be imputed so to create some we eill delete some values and create null values 

In [13]:
order_payments_df.show(2)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 2 rows



In [14]:
payments_df_with_null = order_payments_df.withColumn('payment_value', when(col('payment_value')!=99.33, col('payment_value')).otherwise(lit(None)))

In [15]:
payments_df_with_null.show(2)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 2 rows



In [16]:
imputer= Imputer(inputCols=['payment_value'],outputCols=['payments_df_with_null']).setStrategy('mean')


payments_df_cleaned = imputer.fit(payments_df_with_null).transform( payments_df_with_null)


In [17]:
payments_df_cleaned.show(5)

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payments_df_with_null|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|   154.10987195285352|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
+--------------------+------------------+------------+--------------------+-------------+---------------

# Standardizing the format

In [18]:
def print_schema(dataframe,name):
    print(f'Missing Values in : {name}')
    dataframe.printSchema()

    
dataframes = [customers_df,geolocation_df,order_items_df,order_payments_df,order_reviews_df,orders_df,products_df,sellers_df]
name = ["customers_df","geolocation_df","order_items_df","order_payments_df","order_reviews_df","orders_df","products_df","sellers_df"]
for i in range(0,len(dataframes)):
    print_schema(dataframes[i],name[i])

Missing Values in : customers_df
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

Missing Values in : geolocation_df
root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)

Missing Values in : order_items_df
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)

Missing Values in : order_payments_df
root
 |-- order_id: string (nullable = true)

In [19]:
orders_df_cleaned.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [20]:
order_df_cleaned= order_df_cleaned.withColumn('order_purchase_timestamp',to_date(col('order_purchase_timestamp'))) 

In [21]:
order_df_cleaned.show(3)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|              2017-10-02|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|              2018-07-24|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [22]:
payments_df_cleaned.show(5)

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payments_df_with_null|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|   154.10987195285352|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
+--------------------+------------------+------------+--------------------+-------------+---------------

In [23]:
order_payments_df.groupBy('payment_type').count().show()

+------------+-----+
|payment_type|count|
+------------+-----+
| not_defined|    3|
| credit_card|76795|
|      boleto|19784|
|  debit_card| 1529|
|     voucher| 5775|
+------------+-----+



In [24]:
payments_df_cleaned.groupBy('payment_type').count().show()

+------------+-----+
|payment_type|count|
+------------+-----+
| not_defined|    3|
| credit_card|76795|
|      boleto|19784|
|  debit_card| 1529|
|     voucher| 5775|
+------------+-----+



In [25]:
payments_df_cleaned.withColumn('payment_type',when(col('payment_type')=='credit_card','credit card')
                                               .when(col('payment_type')=='debit_card','debit card')
                                               .when(col('payment_type')=='boleto','bamk transfer')
                                               .when(col('payment_type')=='voucher','voucher')
                                               .when(col('payment_type')=='voucher','undefined')
            
                              
                              
                                                      ).groupBy('payment_type').count().show()

+-------------+-----+
| payment_type|count|
+-------------+-----+
|         NULL|    3|
|  credit card|76795|
|   debit card| 1529|
|      voucher| 5775|
|bamk transfer|19784|
+-------------+-----+



In [26]:
payments_df_cleaned.show()

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payments_df_with_null|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|   154.10987195285352|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               

In [27]:
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [28]:
customers_df.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [29]:
customers_df_cleaned = customers_df.withColumn('customer_zip_code_prefix', col('customer_zip_code_prefix').cast('String'))

In [30]:
customers_df_cleaned.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



# Remove Duplicate Values/Records

In [31]:
customers_df_cleaned = customers_df.dropDuplicates(['customer_id'])

In [32]:
orders_df.show(3)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [33]:
order_with_details = orders_df_cleaned.join(order_items_df,'order_id','left')\
.join(payments_df_cleaned,'order_id','left')\
.join(customers_df,'customer_id','inner')

In [34]:

order_with_details.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+------------------+------------+--------------------+-------------+---------------------+--------------------+------------------------+-------------+--------------+
|         customer_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|payment_sequential|payment_type|payment_installments|payment_value|payments_df_with_null|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+--------------------+--------------------+------------+------------------------+-----------------

In [35]:
order_with_total_value= order_with_details.groupBy('order_id').agg(sum(col('price')).alias('total_price'))
order_with_total_value.show(5)

+--------------------+------------------+
|            order_id|       total_price|
+--------------------+------------------+
|1c4a92d82c1b0dec1...|189.07999999999998|
|28eaf054725f4dd3c...|              29.9|
|78cd965d0bc0388d3...|             169.9|
|126f2d9c30f82426d...|             140.9|
|d6d7c431275f0029d...|             145.0|
+--------------------+------------------+
only showing top 5 rows



In [36]:
# delivery time from prvious notebook
delivery_time = order_with_details.select('price','order_delivered_customer_date','order_purchase_timestamp')\
                     .withColumn('delivery_time_taken',date_diff(col('order_delivered_customer_date'),col('order_purchase_timestamp')))
delivery_time.show()


+------+-----------------------------+------------------------+-------------------+
| price|order_delivered_customer_date|order_purchase_timestamp|delivery_time_taken|
+------+-----------------------------+------------------------+-------------------+
| 29.99|          2017-10-10 21:25:13|     2017-10-02 10:56:33|                  8|
| 29.99|          2017-10-10 21:25:13|     2017-10-02 10:56:33|                  8|
| 29.99|          2017-10-10 21:25:13|     2017-10-02 10:56:33|                  8|
| 118.7|          2018-08-07 15:27:45|     2018-07-24 20:41:37|                 14|
| 159.9|          2018-08-17 18:06:29|     2018-08-08 08:38:49|                  9|
|  45.0|          2017-12-02 00:28:42|     2017-11-18 19:28:06|                 14|
|  19.9|          2018-02-16 18:17:02|     2018-02-13 21:18:39|                  3|
| 147.9|          2017-07-26 10:57:55|     2017-07-09 21:57:05|                 17|
|  49.9|          9999-12-31 00:00:00|     2017-04-11 12:22:08|            2

# Advance Transformation

In [37]:
order_items_df.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [38]:
quantile = order_items_df.approxQuantile('price',[0.25,0.5,0.75],0.0)

In [39]:
q1,q2,q3= quantile[0],quantile[1],quantile[2]
print(f'{q1} {q2} {q3}')

39.9 74.99 134.9


In [40]:
order_items_df.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [41]:
#getting the low and the high cutt off to remove the outliers
quantile =  order_items_df.approxQuantile('price',[0.01,0.99],0)
low_cuttoff,high_cuttoff = quantile[0],quantile[1]
print(f'{low_cuttoff} {high_cuttoff}')

9.99 890.0


In [42]:
order_items_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [43]:
order_items_cleaned = order_items_df.filter((col('price') >= low_cuttoff ) & (col('price') <= high_cuttoff))
order_items_cleaned.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [44]:
order_items_cleaned.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            110453|
|   mean|108.49213068006871|
| stddev|112.87303173792675|
|    min|              9.99|
|    25%|             39.99|
|    50%|              74.9|
|    75%|             130.0|
|    max|             890.0|
+-------+------------------+



In [45]:
products_df.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [46]:
products_df_cleaned  = products_df.withColumn('product_size', 
                      when(col('product_weight_g')<500,'Small')
                       .when(col('product_weight_g')<2000,'Medium')
                       .otherwise('large')
                      
                      )
products_df_cleaned.show(5)


+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_size|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|       Small|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|      Medium|
|96bd76ec8810374ed...|        

In [47]:
products_df_cleaned.groupBy('product_size').count().show()

+------------+-----+
|product_size|count|
+------------+-----+
|      Medium|12505|
|       Small|12464|
|       large| 7982|
+------------+-----+



In [48]:
## caolculationg total revenue per seller

In [49]:
order_items_cleaned.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [50]:
total_seller_revenue = order_items_cleaned.groupBy('seller_id').agg(sum('price').alias('total_revenue'))
total_seller_revenue.orderBy(col('total_revenue').desc()).show(5)

+--------------------+------------------+
|           seller_id|     total_revenue|
+--------------------+------------------+
|4869f7a5dfa277a7d...|228572.63999999908|
|4a3ca9315b744ce9f...| 200472.9199999981|
|7c67e1448b00f6e96...|187923.89000000118|
|fa1c13f2614d7b5c4...| 162793.4399999999|
|da8622b14eb17ae28...| 160196.9699999994|
+--------------------+------------------+
only showing top 5 rows



In [53]:
!hadoop fs -ls /tmp/Brazilian/

Found 9 items
-rw-r--r--   2 harshvardhansahu06733 hadoop    9033957 2025-08-21 20:04 /tmp/Brazilian/olist_customers_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   61273883 2025-08-21 20:04 /tmp/Brazilian/olist_geolocation_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   15438671 2025-08-21 20:04 /tmp/Brazilian/olist_order_items_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop    5777138 2025-08-21 20:04 /tmp/Brazilian/olist_order_payments_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   14451670 2025-08-21 20:04 /tmp/Brazilian/olist_order_reviews_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   17654914 2025-08-21 20:04 /tmp/Brazilian/olist_orders_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop    2379446 2025-08-21 20:04 /tmp/Brazilian/olist_products_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop     174703 2025-08-21 20:04 /tmp/Brazilian/olist_sellers_dataset.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop       2613 2025

In [52]:
!hadoop fs -rm -r /tmp/Brazilian/olist_proc

Deleted /tmp/Brazilian/olist_proc


In [54]:
!hadoop fs -mkdir /tmp/Brazilian/olist_proc

In [55]:
order_with_details.write.mode('overwrite').parquet('/tmp/Brazilian/olist_proc/cleaned_data.parquet')

In [56]:
products_df_cleaned.write.mode('overwrite').parquet('/tmp/Brazilian/olist_proc/products_df_cleaned.parquet')

In [57]:
!hadoop fs -ls /tmp/Brazilian/olist_proc

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-08-23 16:57 /tmp/Brazilian/olist_proc/cleaned_data.parquet
drwxr-xr-x   - root hadoop          0 2025-08-23 16:57 /tmp/Brazilian/olist_proc/products_df_cleaned.parquet


In [58]:
order_with_details.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- payments_df_with_null: double (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- 

In [59]:
products_df_cleaned.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- product_size: string (nullable = false)



In [60]:
#create External Table 
CREATE EXTERNAL TABLE cleaned_products{
    product_id string ,
  product_category_name string ,
  product_name_lenght integer ,
  product_description_lenght integer, 
  product_photos_qty integer ,
  product_weight_g integer ,
  product_length_cm integer,
  product_height_cm integer,
  product_width_cm integer ,
  product_size string 
}
STORED AS PARQUET
LOCATION '/tmp/Brazilian/olist_proc/products_df_cleaned.parquet'


SyntaxError: invalid syntax (2910447226.py, line 2)

In [61]:
spark.sql('show tables').show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
|  default|cleaned_products|      false|
|  default|  customers_1100|      false|
|  default|external_table_2|      false|
+---------+----------------+-----------+



In [63]:
spark.sql('select * from cleaned_products').show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+
|          product_id|product_category_name|product_name_length|product_description_length|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_size|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+
|1e9e8ef04dbcff454...|           perfumaria|               NULL|                      NULL|                 1|             225|               16|               10|              14|       Small|
|3aa071139cb16b67c...|                artes|               NULL|                      NULL|                 1|            1000|               30|               18|              20|      Medium|
|96bd76ec8810374ed...|        

In [ ]:
spark.stop()